## HERE Places API

works with [HERE Places (Search) API](https://developer.here.com/documentation/places/dev_guide/topics_api/resource-explore.html)

Free limit is 250k requests per month, you have to get an API key to replicate the process.

In [22]:
import geopandas as gpd
import pandas as pd
from pyproj import CRS
import folium
import requests
from shapely import wkt

In [23]:
# reading api keys
api_keys = pd.read_excel('../api_keys.xlsx')
api_keys.set_index('key_name', inplace=True)

# HERE Rest api key
here_api_key = api_keys.loc['here_rest_api']['key']

In [24]:
# centroids for city
centroids = gpd.read_file('./input_data/centroid.shp')

print ('Number of cells in grid:',len(centroids))
print ('The CRS for grid layer is correct:', centroids.crs.to_epsg()==4326)

Number of cells in grid: 1724
The CRS for grid layer is correct: True


In [25]:
df_sample = centroids.sample(n=50)

In [26]:
URL = 'https://places.ls.hereapi.com/places/v1/discover/explore'

In [27]:
radius=300

In [28]:
def here_poi_request(lat, lon, category, start_id):

    # start point 
    start_point = f"{lat:.6f},{lon:.6f};r={radius}"
    params = { 
        'apiKey': here_api_key,
        'in': start_point,
        'cat': category,
            }
    response = requests.get(URL, params=params)
    response_json = response.json()
    try:
        poi = pd.json_normalize(response_json['results']['items'])
        
        # coordinates conversion
        coord = poi['position'].astype('str').str.strip('[]').str.split(", ", expand = True)
        coord.columns = ['lat', 'lon']

        coord['lat'] = coord['lat'].astype('float64')
        coord['lon'] = coord['lon'].astype('float64')

        poi = poi.join(coord)

        # start point
        poi['start_id'] = start_id
        
        return poi
    
    except Exception as e:
        print ('for point', lat, lon)
        print ('result is')
        print (response_json['results']['items'])

categories in HERE places search can be explored on https://developer.here.com/documentation/places/dev_guide/topics/place_categories/places-category-system.html

In [29]:
category = 'eat-drink'

poi_long_list = pd.DataFrame()

point_lat_list = df_sample['lat'].to_list()
point_lon_list = df_sample['lon'].to_list()
start_ids_list = df_sample['id'].to_list()


for lat, lon, start_id in zip(point_lat_list, point_lon_list, start_ids_list):
    try:
        poi_part = here_poi_request(lat, lon, category, start_id)
        poi_long_list = poi_long_list.append(poi_part)
    except Exception as e:
        print (e)

for point 54.90051009362103 52.422380831632005
result is
[]
for point 54.948170026860296 52.38112449898957
result is
[]
for point 54.87893735520465 52.3436842244789
result is
[]
for point 54.86308534695454 52.35484287100544
result is
[]
for point 54.93372099589962 52.26356640624879
result is
[]
for point 54.878186756664086 52.40990409830355
result is
[]
for point 54.89144386004941 52.2232629552624
result is
[]
for point 54.948081118481184 52.38892849181378
result is
[]
for point 54.8797742072568 52.26577466384227
result is
[]
for point 54.93720376938205 52.35733384688641
result is
[]
for point 54.85522826584042 52.24942793498776
result is
[]
for point 54.878544474879575 52.378742187516316
result is
[]
for point 54.87266558925483 52.300617449087156
result is
[]
for point 54.91823929475309 52.23967787559933
result is
[]
for point 54.89136513790634 52.23105646950997
result is
[]
for point 54.91029518385178 52.352528332287
result is
[]
for point 54.86143521058681 52.30025595791929
result i

In [30]:
len(poi_long_list)

35

In [31]:
poi_long_list['id'].nunique()

33

In [32]:
#extra columns
#poi_long_list.drop(['icon', 'averageRating', 
       #          'openingHours.text', 'openingHours.label', 'openingHours.isOpen', 'openingHours.structured'], 
           #                 axis=1, inplace = True)

# drop duplicates
poi_long_list.drop_duplicates(subset='id', inplace=True)
len(poi_long_list)

33

In [34]:
poi_long_list.to_csv('./output/{}.csv'.format(category))